In [12]:
from gensim.models import Word2Vec
import numpy as np

In [22]:
class Preprocessing:
    end_ch=':.!";'
    punct = '!.:;,?"\'-'
    numbers = '1234567890{}()'
    lat_letters = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    
    def readline(file, delimiters):
        buffer = ''
        while True:
            b = file.read(1)
            buffer += b
            if b:
                if b in delimiters:
                   yield buffer
                   buffer = ''
            else:
               yield buffer
               break
    
    def load_pushkin(filename='./Корпуса/Пушкин.txt'):
        pushkin_file = open(filename, 'r')
        pushkin_phrases = list()
        phrase=''
        for line in pushkin_file.readlines():
            line="".join(ch for ch in line if ch not in ('"', "'", '\t')).strip()
            phrase+=' '+line
            if len(line):
                if ((Preprocessing.end_ch+',').find(line[-1])!=-1):
                    pushkin_phrases.append(phrase.strip()[:-1]+'.')
                    phrase=''
                else:
                    phrase = line
        pushkin_file.close()
        return pushkin_phrases
    
    def parse_pushkin(pushkin_phrases):
        p_sentences=list()
        for phrase in pushkin_phrases:
                words = "".join(ch for ch in phrase 
                                if ch not in Preprocessing.punct+Preprocessing.numbers)\
                .lower().split()
                words = list(filter(lambda w: len(w)>2, words))
                p_sentences.append(words)
        return p_sentences

    def load_tolstoy(filename='./Корпуса/Война и Мир.txt'):
        tolstoy_file = open(filename, 'r')
        sentences=list()
        for line in Preprocessing.readline(tolstoy_file, Preprocessing.end_ch):
            line = line.strip()
            line = "".join(ch for ch in line if ch not in Preprocessing.punct)
            if len(line):
                words = line.split()
                words = list(filter(lambda w: len(w)>2, words))
                sentences.append(words)
        tolstoy_file.close()
        return sentences

    def load_dialog(filename='./Корпуса/Диалоги.txt'):
        dialogs_file = open(filename, 'r', encoding='utf-8')
        sentences=list()
        for line in Preprocessing.readline(dialogs_file, "\n"):
            line = line.strip()[2:]
            line = "".join(ch for ch in line if ch not in Preprocessing.punct)
            if len(line):
                words = line.split()
                words = list(filter(lambda w: len(w)>2, words))
                sentences.append(words)
        dialogs_file.close()
        return sentences

In [40]:
class Model:
    def __init__(self, learning_text, _size, _window):
        self.w2v_model = Word2Vec(learning_text, size = _size, window = _window)
    
    def train(self, learning_phrases, numbers, _epochs):
        model.train(learning_phrases, total_examples = numbers, epochs = _epochs)
        
    def set_phrase(self, _phrase):
        self.phrase = _phrase
        
    def get_phrase(self):
        return self.phrase
    
    def get_model_words(self):
        return list(self.w2v_model.wv.vocab.keys())

In [52]:
class Conversation_with_Poet:
    pushkin_phrases = Preprocessing.load_pushkin()
    pushkin_sentences = Preprocessing.parse_pushkin(pushkin_phrases)
    tolstoy_sentences = Preprocessing.load_tolstoy()
    dialog_sentences = Preprocessing.load_dialog()
    model = Model(pushkin_sentences+tolstoy_sentences+dialog_sentences, 300, 5)
    
    def get():
        return Conversation_with_Poet.model.get_phrase()
    
    def post(phrase):
        phrase_list=list()
        input_phrase = "".join\
        (ch for ch in phrase if ch not in Preprocessing.punct+Preprocessing.numbers)\
        .lower().split()
        sentence_sim = dict()
        i=0
        for sentence in Conversation_with_Poet.pushkin_sentences:
            learning_phrase = list(filter(lambda w: len(w)>2, input_phrase))
            
            sentence = list(filter(lambda w: w in\
                                   Conversation_with_Poet.model.get_model_words(),\
                                   sentence))
            input_phrase = list(filter(lambda w: w in\
                                      Conversation_with_Poet.model.get_model_words(),\
                                      input_phrase))
            if len(sentence)!=0 and len(input_phrase)!=0:
                sentence_sim[Conversation_with_Poet.pushkin_phrases[i]] = \
                Conversation_with_Poet.model.w2v_model.wv.n_similarity(sentence, input_phrase)
            i+=1
        try:
            max_sim = max(sentence_sim.values())
        except:
            max_sim=0
        for phrase in Conversation_with_Poet.pushkin_phrases:
            if sentence_sim.get(phrase)==max_sim:
                phrase_list.append(phrase)
        Conversation_with_Poet.model.set_phrase(np.random.choice(phrase_list))

In [55]:
Conversation_with_Poet.post("Может быть не надо было идти на дуэль с Дантесом?")

In [56]:
Conversation_with_Poet.get()

'Пора, пора мне быть умней.'